In [1]:
from openai import OpenAI
from typing_extensions import override
from openai import AssistantEventHandler
import json

client = OpenAI()

In [2]:
def get_medicine_availability(params):
  name = params.get('name')
  return f'Hay las que quieras. {name}'

In [3]:
def actions(action_name, params):
  if action_name == 'get_medicine_availability':
    return get_medicine_availability(params)
  return 'No se que hacer'

In [4]:
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

In [5]:
assistant = client.beta.assistants.retrieve(assistant_id='asst_j73v41e4e1pDzwN2ca513DA1')

In [6]:
def create_thread():
  thread = client.beta.threads.create()
  return thread

In [11]:
def add_message(thread, content):
  message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content=content
  )
  
  return message

In [15]:
def get_answer(message, thread):
  with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    event_handler=EventHandler(),
  ) as stream:
    stream.until_done()
    
  final_run = stream.get_final_run()
  status = final_run.status

  if status == 'requires_action':

    function_name = final_run.required_action.submit_tool_outputs.tool_calls[0].function.name
    arguments = final_run.required_action.submit_tool_outputs.tool_calls[0].function.arguments
    response = actions(function_name, json.loads(arguments))
    print(f'assistant > {response}')
  else:
    print(f'assistant > {final_run}')

In [16]:
thread = create_thread()

while True:
  input_message = input('user > ')
  
  if 'exit' in input_message:
    break
  
  message = add_message(thread, input_message)
  get_answer(message, thread)


assistant > file_search


assistant > Con los medicamentos vencidos se deben seguir ciertas pautas para su manejo adecuado. Según la Resolución No. 0371 del 26 de febrero de 2009, se establece que:

1. **Devolución**: Los consumidores deben devolver los medicamentos vencidos a los mecanismos de devolución establecidos por el fabricante o importador. Esto incluye seguir las instrucciones de manejo seguro que se encuentran en la etiqueta del producto【4:1†source】.

2. **Centros de Acopio**: Se deben establecer centros de acopio donde los consumidores puedan depositar los medicamentos vencidos. Estos centros deben cumplir con requisitos de seguridad y estar debidamente autorizados【4:13†source】.

3. **Prohibiciones**: Está prohibido disponer de medicamentos vencidos en rellenos sanitarios, quemarlos, verterlos en cuerpos de agua o abandonarlos en espacios públicos. Solo se pueden manejar en instalaciones autorizadas para su tratamiento y disposición final【4:2†source】【4:13†source】.

4. **In